<a href="https://colab.research.google.com/github/srihari1306/SafeVisionAI/blob/srihari/Colab_model_training_notebooks%5CCCTV_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# CELL 1: SETUP - Mount Drive, Unzip Dataset, Install Dependencies
# ============================================================================

print("=" * 70)
print("CELL 1: ENVIRONMENT SETUP")
print("=" * 70)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required libraries
print("\n[INFO] Installing required libraries...")
!pip install -q opencv-python-headless tqdm scikit-learn

# Import libraries
import os
import zipfile
import shutil
from pathlib import Path

# Define paths
DATASET_ZIP = '/content/drive/MyDrive/archive.zip'
EXTRACT_TO = '/content/hwid12_data'
MODEL_SAVE_PATH = '/content/drive/MyDrive/accident_video_model.h5'

# Unzip dataset
print("\n[INFO] Extracting dataset...")
if os.path.exists(EXTRACT_TO):
    shutil.rmtree(EXTRACT_TO)

os.makedirs(EXTRACT_TO, exist_ok=True)

with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_TO)

print(f"[SUCCESS] Dataset extracted to: {EXTRACT_TO}")

# List extracted folders
print("\n[INFO] Dataset structure:")
for root, dirs, files in os.walk(EXTRACT_TO):
    level = root.replace(EXTRACT_TO, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    if level < 2:  # Only show first 2 levels
        subindent = ' ' * 2 * (level + 1)
        for dir_name in dirs[:5]:  # Show first 5 folders
            print(f'{subindent}{dir_name}/')
        if len(dirs) > 5:
            print(f'{subindent}... and {len(dirs) - 5} more folders')
    if level == 0:
        break

print("\n" + "=" * 70)
print("SETUP COMPLETE!")
print("=" * 70)

CELL 1: ENVIRONMENT SETUP
Mounted at /content/drive

[INFO] Installing required libraries...

[INFO] Extracting dataset...
[SUCCESS] Dataset extracted to: /content/hwid12_data

[INFO] Dataset structure:
hwid12_data/
  Video-Accident-Dataset/

SETUP COMPLETE!


In [ ]:
# ============================================================================
# CELL A: PREPROCESS & CACHE VIDEO FRAMES (OPTIMIZED)
# ============================================================================

import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import glob


DATASET_ROOT = "/content/hwid12_data"
CACHE_DIR = "/content/processed_frames"
SEQUENCE_LENGTH = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

os.makedirs(CACHE_DIR, exist_ok=True)

records = []

print("[INFO] Scanning dataset...")

video_files = []
video_files += glob.glob(os.path.join(DATASET_ROOT, "**", "*.mp4"), recursive=True)
video_files += glob.glob(os.path.join(DATASET_ROOT, "**", "*.avi"), recursive=True)

print(f"[INFO] Found {len(video_files)} total videos. Starting processing...")

for vid_path in tqdm(video_files):
    try:
        folder_name = os.path.basename(os.path.dirname(vid_path)).lower()
        if "negative" in folder_name or "normal" in folder_name:
            label = 0
        else:
            label = 1

        cap = cv2.VideoCapture(vid_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if total_frames <= SEQUENCE_LENGTH:
            indices = np.arange(total_frames)
        else:
            indices = np.linspace(0, total_frames - 1, SEQUENCE_LENGTH).astype(int)

        frames = []
        current_frame = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if current_frame in indices:
                frame = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)

            current_frame += 1
            # Optimization: Stop reading if we have enough frames
            if len(frames) == len(indices):
                break

        cap.release()

        # Convert to numpy array
        # CRITICAL: Use uint8 (0-255) to save disk space!
        frames = np.array(frames, dtype=np.uint8)

        # Handle Padding (if video was short)
        if len(frames) < SEQUENCE_LENGTH:
            padding = np.zeros((SEQUENCE_LENGTH - len(frames), IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.uint8)
            frames = np.concatenate([frames, padding], axis=0)

        # Truncate (safety)
        frames = frames[:SEQUENCE_LENGTH]

        # Generate unique filename
        # Hash or index based name is safer than original filename to avoid duplicates
        file_name = f"{len(records)}_{label}.npy"
        save_path = os.path.join(CACHE_DIR, file_name)

        np.save(save_path, frames)
        records.append([save_path, label])

    except Exception as e:
        print(f"[ERROR] Failed {vid_path}: {e}")

# Save Metadata
df = pd.DataFrame(records, columns=["npy_path", "label"])
df.to_csv(os.path.join(CACHE_DIR, "metadata.csv"), index=False)

print(f"\n[SUCCESS] Processed {len(df)} videos.")
print(f"Frames saved as uint8 at: {CACHE_DIR}")

[INFO] Scanning dataset...
[INFO] Found 2782 total videos. Starting processing...


100%|██████████| 2782/2782 [44:31<00:00,  1.04it/s]


[SUCCESS] Processed 2782 videos.
Frames saved as uint8 at: /content/processed_frames


In [ ]:
# ============================================================================
# CELL B: FAST .NPY DATA GENERATOR
# ============================================================================

import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

CACHE_DIR = "/content/processed_frames"

# Check if metadata exists
if not os.path.exists(f"{CACHE_DIR}/metadata.csv"):
    raise FileNotFoundError("Metadata not found! Run Cell A first.")

df = pd.read_csv(f"{CACHE_DIR}/metadata.csv")

# Stratified Split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
val_df, test_df = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"]
)
print(f"Training Samples: {len(train_df)}")
print(f"Validation Samples: {len(val_df)}")
print(f"Testing Samples: {len(test_df)}")

class FastNumpyGenerator(keras.utils.Sequence):
    def __init__(self, df, batch_size=16, shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        idxs = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[idxs]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            # Load uint8 array (0-255)
            data = np.load(row["npy_path"])

            # Normalize to float32 (0-1) HERE to save memory during storage
            data = data.astype('float32') / 255.0

            X.append(data)
            y.append(row["label"])

        return np.array(X), np.array(y)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Note: Batch size can be higher now because reading .npy is efficient
train_generator = FastNumpyGenerator(train_df, batch_size=16, shuffle=True)
val_generator = FastNumpyGenerator(val_df, batch_size=16, shuffle=False)
test_generator = FastNumpyGenerator(test_df, batch_size=16, shuffle=False)

print("[INFO] FAST Generators ready!")

Training Samples: 2225
Validation Samples: 278
Testing Samples: 279
[INFO] FAST Generators ready!


In [ ]:
# ============================================================================
# CELL 3: MODEL TRAINING (FIXED INPUT SHAPE)
# ============================================================================

print("=" * 70)
print("CELL 3: MODEL TRAINING")
print("=" * 70)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

SEQUENCE_LENGTH = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224
MODEL_SAVE_PATH = '/content/drive/MyDrive/accident_video_model.h5'

def create_model():
    input_layer = layers.Input(shape=(SEQUENCE_LENGTH, IMG_HEIGHT, IMG_WIDTH, 3))

    cnn = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        pooling='avg'
    )
    cnn.trainable = False

    x = layers.TimeDistributed(cnn)(input_layer)

    x = layers.LSTM(64, return_sequences=False)(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=input_layer, outputs=outputs)
    return model

model = create_model()

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

print("[INFO] Model built with input shape:", model.input_shape)

callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_loss', verbose=1),
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

CELL 3: MODEL TRAINING
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[INFO] Model built with input shape: (None, 50, 224, 224, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7491 - auc: 0.8097 - loss: 0.5191
Epoch 1: val_loss improved from inf to 0.15544, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 544s 2s/step - accuracy: 0.7497 - auc: 0.8105 - loss: 0.5181 - val_accuracy: 0.9659 - val_auc: 0.9929 - val_loss: 0.1554 - learning_rate: 1.0000e-04
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9435 - auc: 0.9854 - loss: 0.1873
Epoch 2: val_loss improved from 0.15544 to 0.11462, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.9435 - auc: 0.9854 - loss: 0.1871 - val_accuracy: 0.9659 - val_auc: 0.9930 - val_loss: 0.1146 - learning_rate: 1.0000e-04
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9620 - auc: 0.9902 - loss: 0.1249
Epoch 3: val_loss improved from 0.11462 to 0.08260, saving model to /content/drive/MyDrive/accident_video_model.h5


...

140/140 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - accuracy: 0.9721 - auc: 0.9946 - loss: 0.0983 - val_accuracy: 0.9785 - val_auc: 0.9975 - val_loss: 0.0665 - learning_rate: 1.0000e-04
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9768 - auc: 0.9939 - loss: 0.0819
Epoch 5: val_loss improved from 0.06651 to 0.06594, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 304s 2s/step - accuracy: 0.9768 - auc: 0.9939 - loss: 0.0818 - val_accuracy: 0.9785 - val_auc: 0.9974 - val_loss: 0.0659 - learning_rate: 1.0000e-04
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9799 - auc: 0.9964 - loss: 0.0644
Epoch 6: val_loss improved from 0.06594 to 0.06333, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.9799 - auc: 0.9964 - loss: 0.0644 - val_accuracy: 0.9749 - val_auc: 0.9983 - val_loss: 0.0633 - learning_rate: 1.0000e-04
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9846 - auc: 0.9957 - loss: 0.0650
Epoch 7: val_loss did not improve from 0.06333
140/140 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.9847 - auc: 0.9957 - loss: 0.0649 - val_accuracy: 0.9767 - val_auc: 0.9969 - val_loss: 0.0702 - learning_rate: 1.0000e-04
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9842 - auc: 0.9982 - loss: 0.0486
Epoch 8: val_loss improved from 0.06333 to 0.06305, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 303s 2s/step - accuracy: 0.9842 - auc: 0.9982 - loss: 0.0486 - val_accuracy: 0.9803 - val_auc: 0.9980 - val_loss: 0.0630 - learning_rate: 1.0000e-04
Epoch 9/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9884 - auc: 0.9986 - loss: 0.0405
Epoch 9: val_loss did not improve from 0.06305
140/140 ━━━━━━━━━━━━━━━━━━━━ 290s 2s/step - accuracy: 0.9884 - auc: 0.9987 - loss: 0.0405 - val_accuracy: 0.9785 - val_auc: 0.9986 - val_loss: 0.0671 - learning_rate: 1.0000e-04
Epoch 10/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9881 - auc: 0.9971 - loss: 0.0427
Epoch 10: val_loss improved from 0.06305 to 0.05630, saving model to /content/drive/MyDrive/accident_video_model.h5


140/140 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.9881 - auc: 0.9971 - loss: 0.0427 - val_accuracy: 0.9803 - val_auc: 0.9968 - val_loss: 0.0563 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 10.


In [ ]:
from tensorflow.keras.models import load_model

MODEL_SAVE_PATH = "/content/drive/MyDrive/accident_video_model.h5"

print("Loading trained model from:", MODEL_SAVE_PATH)
model = load_model(MODEL_SAVE_PATH)

print("[INFO] Model loaded successfully!")
print("Model Input Shape:", model.input_shape)

Loading trained model from: /content/drive/MyDrive/accident_video_model.h5


[INFO] Model loaded successfully!
Model Input Shape: (None, 50, 224, 224, 3)


In [ ]:
# Save final model
model.save(MODEL_SAVE_PATH)
print(f"\n[SUCCESS] Model saved to: {MODEL_SAVE_PATH}")

# Training results
print("\n" + "=" * 70)
print("TRAINING COMPLETE!")
print("=" * 70)
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")


[SUCCESS] Model saved to: /content/drive/MyDrive/accident_video_model.h5

TRAINING COMPLETE!
Final Training Accuracy: 0.9888
Final Validation Accuracy: 0.9803
Final Training Loss: 0.0389
Final Validation Loss: 0.0563


In [ ]:
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

print("\n" + "="*70)
print("MODEL EVALUATION ON TEST SET")
print("="*70)

y_true = test_df["label"].values

start_time = time.time()
y_pred_probs = model.predict(test_generator)
end_time = time.time()

y_pred = (y_pred_probs > 0.5).astype(int).flatten()

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

try:
    roc_auc = roc_auc_score(y_true, y_pred_probs)
except:
    roc_auc = "Undefined (only one class present)"

num_samples = len(test_generator.df)
total_time = end_time - start_time
avg_latency = (total_time / num_samples) * 1000

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc}")
print(f"Average Inference Latency: {avg_latency:.2f} ms/video")



MODEL EVALUATION ON TEST SET


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 157s 5s/step
Accuracy: 0.9857
Precision: 0.9939
Recall: 0.9820
F1-Score: 0.9880
ROC-AUC: 0.9990911035072711
Average Inference Latency: 734.96 ms/video


In [ ]:
# ============================================================================
# CELL 4: INFERENCE ON CUSTOM VIDEO (FIXED RESOLUTION)
# ============================================================================

print("=" * 70)
print("CELL 4: VIDEO INFERENCE & DETECTION")
print("=" * 70)

import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from collections import deque
from google.colab import files

TEST_VIDEO_PATH = '/content/drive/MyDrive/demo.mp4'
OUTPUT_VIDEO_PATH = '/content/output_detected.mp4'
MODEL_PATH = '/content/drive/MyDrive/accident_video_model.h5'

SEQUENCE_LENGTH = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224
CONFIDENCE_THRESHOLD = 0.5


def create_mobilenet_lstm_model(sequence_length=50, img_height=224, img_width=224):
    input_layer = layers.Input(shape=(sequence_length, img_height, img_width, 3))

    mobilenet = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(img_height, img_width, 3),
        pooling='avg'
    )
    mobilenet.trainable = False

    x = layers.TimeDistributed(mobilenet)(input_layer)
    x = layers.LSTM(64, return_sequences=False)(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    output_layer = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=input_layer, outputs=output_layer)
    return model

print("\n[INFO] Loading trained model...")
try:
    model = keras.models.load_model(MODEL_PATH)
    print("[SUCCESS] Model loaded directly")
except:
    print("[INFO] Recreating model architecture...")
    model = create_mobilenet_lstm_model(SEQUENCE_LENGTH, IMG_HEIGHT, IMG_WIDTH)
    model.load_weights(MODEL_PATH)
    print("[SUCCESS] Model weights loaded")

if not os.path.exists(TEST_VIDEO_PATH):
    print(f"\n[WARNING] Test video not found at: {TEST_VIDEO_PATH}")
    print("[INFO] Please upload your test video to Google Drive or update the path")
else:
    print(f"\n[INFO] Processing video: {TEST_VIDEO_PATH}")

    cap = cv2.VideoCapture(TEST_VIDEO_PATH)

    if not cap.isOpened():
        print("[ERROR] Failed to open video file")
    else:
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"[INFO] Video properties:")
        print(f"   Resolution: {width}x{height}")
        print(f"   FPS: {fps}")
        print(f"   Total frames: {total_frames}")

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (width, height))

        frame_queue = deque(maxlen=SEQUENCE_LENGTH)

        print("\n[INFO] Processing frames...")
        from tqdm import tqdm
        progress_bar = tqdm(total=total_frames, desc="Processing")

        frame_count = 0
        detection_label = "BUFFERING..."
        detection_color = (0, 255, 255)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            processed_frame = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
            processed_frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
            processed_frame_rgb = processed_frame_rgb.astype(np.float32) / 255.0

            frame_queue.append(processed_frame_rgb)

            if len(frame_queue) == SEQUENCE_LENGTH:
                sequence = np.array(list(frame_queue))
                sequence = np.expand_dims(sequence, axis=0)

                if frame_count % 5 == 0:
                    prediction = model.predict(sequence, verbose=0)[0][0]

                    if prediction > CONFIDENCE_THRESHOLD:
                        detection_label = f"ACCIDENT! ({prediction:.0%})"
                        detection_color = (0, 0, 255)  # Red
                    else:
                        detection_label = f"NORMAL ({prediction:.0%})"
                        detection_color = (0, 255, 0)  # Green

            cv2.rectangle(frame, (0, 0), (width, 60), (0, 0, 0), -1)
            cv2.putText(
                frame,
                detection_label,
                (20, 45),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.0,
                detection_color,
                2,
                cv2.LINE_AA
            )

            out.write(frame)
            frame_count += 1
            progress_bar.update(1)

        progress_bar.close()

        cap.release()
        out.release()

        print(f"\n[SUCCESS] Processed {frame_count} frames")
        print(f"[SUCCESS] Output saved to: {OUTPUT_VIDEO_PATH}")

        print("\n[INFO] Downloading output video...")
        try:
            files.download(OUTPUT_VIDEO_PATH)
            print("[SUCCESS] Video downloaded to your computer!")
        except Exception as e:
            print(f"[WARNING] Auto-download failed: {str(e)}")
            print(f"[INFO] You can manually download from: {OUTPUT_VIDEO_PATH}")

print("\n" + "=" * 70)
print("INFERENCE COMPLETE!")
print("=" * 70)

CELL 4: VIDEO INFERENCE & DETECTION

[INFO] Loading trained model...


[SUCCESS] Model loaded directly

[INFO] Processing video: /content/drive/MyDrive/demo.mp4
[INFO] Video properties:
   Resolution: 848x382
   FPS: 24
   Total frames: 268

[INFO] Processing frames...


Processing: 100%|██████████| 268/268 [01:19<00:00,  3.37it/s]


[SUCCESS] Processed 268 frames
[SUCCESS] Output saved to: /content/output_detected.mp4

[INFO] Downloading output video...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[SUCCESS] Video downloaded to your computer!

INFERENCE COMPLETE!
